In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from nltk.tokenize import RegexpTokenizer
from preprocess import preprocess
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv(os.path.join("data", "train.csv"))
test_data = pd.read_csv(os.path.join("data", "test.csv"))
train_data.dropna(inplace=True)
X, y = train_data.drop(["price"], axis=1), train_data["price"]
y = y.to_numpy().astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_test = test_data

X_train = preprocess(X_train).to_numpy().astype(float)
X_val = preprocess(X_val).to_numpy().astype(float)
X_test = preprocess(X_test)

# scale
scaler =  RobustScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape

(11740, 349)

In [4]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [10, 30, 50, 70],
    'learning_rate' : [0.01, 0.03, 0.1, 0.3]
}

# Adaboost with DecisionTree as base estimator
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier()
)

gscv = GridSearchCV(ada_clf, param_grid, cv=5, scoring="f1")
gscv.fit(X_train, y_train)
ada_clf = gscv.best_estimator_
# ada_test_accuracy = ada_clf.score(X_val, y_val)
# print("Adaboost Test Accuracy:", ada_test_accuracy)

/home/muchang/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/muchang/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/muchang/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/muchang/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/muchang/.local/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/home/muchang/.local/lib/python3.8/site-packages/sklearn/m

KeyboardInterrupt: 

In [6]:
import xgboost as xgb

param_grid = {
    'n_estimators' : [10, 30, 50, 70],
    'learning_rate' : [0.1, 0.3, 0.1, 0.3, 1.0, 3.0]
}

# XGBoost Classifier
xgb_clf = xgb.XGBClassifier(
    objective="multi:softmax"
)
gscv2 = GridSearchCV(xgb_clf, param_grid, cv=5, verbose=3, n_jobs=20, scoring="accuracy")
gscv2.fit(X_train, y_train)
xgb_clf = gscv2.best_estimator_
# Test accuracy
xgb_test_accuracy = xgb_clf.score(X_val, y_val)
print("XGBoost Val Accuracy:", xgb_test_accuracy)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[CV 2/5] END learning_rate=0.01, n_estimators=10;, score=0.494 total time=   0.7s
[CV 1/5] END learning_rate=0.01, n_estimators=10;, score=0.487 total time=   0.8s
[CV 5/5] END learning_rate=0.01, n_estimators=10;, score=0.465 total time=   1.0s
[CV 3/5] END learning_rate=0.01, n_estimators=10;, score=0.482 total time=   1.0s
[CV 4/5] END learning_rate=0.01, n_estimators=10;, score=0.492 total time=   1.0s
[CV 1/5] END learning_rate=0.03, n_estimators=10;, score=0.500 total time=   0.7s
[CV 2/5] END learning_rate=0.03, n_estimators=10;, score=0.501 total time=   0.7s
[CV 1/5] END learning_rate=0.01, n_estimators=30;, score=0.499 total time=   1.7s
[CV 5/5] END learning_rate=0.01, n_estimators=30;, score=0.469 total time=   1.7s
[CV 3/5] END learning_rate=0.01, n_estimators=30;, score=0.492 total time=   1.7s
[CV 3/5] END learning_rate=0.03, n_estimators=10;, score=0.488 total time=   0.7s
[CV 4/5] END learning_rate=0.01, n_estimators=30;, score=0.497 total time=   1.7s
[CV 5/5] END lea

In [10]:
param_grid = {
    'n_estimators' : [10, 30, 50, 70],
    'learning_rate' : [0.1, 0.3, 0.1, 0.3, 1.0, 3.0], 
    'eval_metric' : ["mlogloss", "merror", "auc"]
}

# XGBoost Classifier
xgb_clf = xgb.XGBClassifier(
    objective="multi:softmax"
)
gscv2 = GridSearchCV(xgb_clf, param_grid, cv=5, verbose=3, n_jobs=20, scoring="accuracy")
gscv2.fit(X_train, y_train)
xgb_clf = gscv2.best_estimator_
# Test accuracy
xgb_test_accuracy = xgb_clf.score(X_val, y_val)
print("XGBoost Val Accuracy:", xgb_test_accuracy)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 3/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=10;, score=0.514 total time=   0.7s
[CV 4/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=10;, score=0.505 total time=   0.7s
[CV 1/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=10;, score=0.515 total time=   0.9s
[CV 2/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=10;, score=0.508 total time=   0.9s
[CV 5/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=10;, score=0.495 total time=   1.0s
[CV 1/5] END eval_metric=mlogloss, learning_rate=0.3, n_estimators=10;, score=0.526 total time=   0.7s
[CV 2/5] END eval_metric=mlogloss, learning_rate=0.3, n_estimators=10;, score=0.511 total time=   0.7s
[CV 4/5] END eval_metric=mlogloss, learning_rate=0.3, n_estimators=10;, score=0.519 total time=   0.7s
[CV 2/5] END eval_metric=mlogloss, learning_rate=0.1, n_estimators=30;, score=0.516 total time=   1.7s
[CV 3/5] EN

In [11]:
X_test_input = X_test.to_numpy().astype(float)
X_test_input = scaler.transform(X_test_input)
test_predictions = xgb_clf.predict(X_test_input)

submission = {
    "id" : list(range(len(test_predictions))), 
    "price" : list(test_predictions.astype(float).astype(int).astype(float))
}

submission = pd.DataFrame.from_dict(submission)

submission.to_csv(os.path.join("submissions/", "bahng_xgb2.csv"), index=False)